# Training an MLP using UKF/EKF

In [844]:
import jax
import flax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
from flax import linen as nn
from jax.random import PRNGKey, normal, split, uniform
from jax.ops import index_update
from tqdm.notebook import tqdm
from functools import partial

In [2]:
%config InlineBackend.figure_format = "retina"
plt.rcParams["axes.spines.top"] = False
plt.rcParams["axes.spines.right"] = False

$$
    y=x + 2\cos(x)(-5)\sin(x) + x^3
$$

In [3]:
def f(x): return x -10 * jnp.cos(x) * jnp.sin(x) + x ** 3

In [1870]:
n_obs = 200
key = PRNGKey(314)
key_x, key_y = split(key, 2)
noise_y = normal(key_y, (n_obs, ))

x = jnp.linspace(-3, 3, n_obs)
x = x + normal(key_x, (n_obs,)) * 0.1
y = f(x) + noise_y * 3

X = np.c_[x, y]
np.random.shuffle(X)
x, y = jnp.array(X.T)

In [1871]:
plt.scatter(x, y, s=14)

## MLP

In [1872]:
class MLP1D(nn.Module):
    n_hidden: int
        
    def setup(self):
        self.layer1 = nn.Dense(self.n_hidden)
        self.layer2 = nn.Dense(self.n_hidden)
        self.layer_out = nn.Dense(1)
    
    def __call__(self, x):
        x = x[:, None]
        x = nn.tanh(self.layer1(x))
#         x = nn.sigmoid(self.layer2(x))
        x = self.layer_out(x)
        return x

In [1873]:
def make_mse(x_batch, y_batch):
    def mse(params):
        def squared_error(x, y):
            yhat = model.apply(params, x)
            err = y - yhat
            return jnp.inner(err, err) / 2.0
        return jnp.mean(jax.vmap(squared_error)(x_batch, y_batch), axis=0).ravel()[0]
    return jax.jit(mse)

loss = make_mse(x[:,None], y)

In [1874]:
key = PRNGKey(314)
key1, key2 = split(key, 2)

model = MLP1D(3)
params = model.init(key1, x) 
yhat = model.apply(params, x)

In [1875]:
yhat = model.apply(params, x)
err = (y[:, None] - yhat)
jnp.linalg.norm(err, 2) ** 2 / (2 * len(x))

DeviceArray(110.32031, dtype=float32)

In [1876]:
alpha = 0.1
n_epochs = 1000
grad_fn = jax.value_and_grad(loss)
pb = tqdm(range(n_epochs))
for i in pb:
    loss_val, grads =  grad_fn(params)
    params = jax.tree_multimap(lambda p, g: p - alpha * g, params, grads)
    if i % 100 == 0:
        desc = f"Loss step {i:03}: {loss_val:0.2f}"
        pb.set_description_str(desc)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [1877]:
xtest = jnp.linspace(x.min(), x.max(), 200)
yhat_test = model.apply(params, xtest)

In [1878]:
plt.plot(xtest, yhat_test, c="tab:red", label="MLP")
# plt.plot(x, f(x), label="underlying", linestyle="--")
plt.scatter(x, y, s=14, c="none", edgecolor="black", label="observations")
plt.title(f"MLP fit after {n_epochs} epochs")
plt.legend()

In [ ]:
# This doesn't work
Hw = jax.jacobian(model)
Hw(x)

# UKF Method

In [1880]:
W0 = jax.tree_map(lambda p: p * 0, params)

In [1881]:
jax.tree_map(lambda p: p.shape, params)

FrozenDict({
    params: {
        layer1: {
            bias: (3,),
            kernel: (1, 3),
        },
        layer_out: {
            bias: (1,),
            kernel: (3, 1),
        },
    },
})

In [1882]:
def fz(W): return W
def fx(W, x): return model.apply(params, x)

In [1883]:
fx(params, x[0, None])

DeviceArray([[1.02888]], dtype=float32)

In [1884]:
leafs, treedef = jax.tree_util.tree_flatten(params)

In [1885]:
jax.tree_unflatten(treedef, leafs)

FrozenDict({
    params: {
        layer1: {
            bias: DeviceArray([-10.06236, -11.09292,   6.91377], dtype=float32),
            kernel: DeviceArray([[ 5.49244, -4.63497,  4.14535]], dtype=float32),
        },
        layer_out: {
            bias: DeviceArray([-1.87585], dtype=float32),
            kernel: DeviceArray([[11.97218],
                         [-7.16364],
                         [ 7.72491]], dtype=float32),
        },
    },
})

### Flatten and reconstruction of the weights

In [1886]:
leafs, treedef = jax.tree_util.tree_flatten(params)
shapes = [leaf.shape for leaf in leafs]

# Flattening of the parameters
all_params = jax.tree_map(lambda v: v.reshape(-1, 1), leafs)
all_params = jnp.concatenate(all_params)
all_params.shape 

(10, 1)

In [1887]:
# cut-off points of the flattened weigths
indices = jnp.array([np.prod(s) for s in shapes]).cumsum()

# Pytree (dict) reconstruction of the weights
split_reconstruction = jnp.array_split(all_params, indices)
split_reconstruction = [arr.reshape(shape) for arr, shape in zip(split_reconstruction, shapes)]
params_reconstructed = jax.tree_unflatten(treedef, split_reconstruction)

In [1888]:
def sqrtm(M):
    """
    Compute the matrix square-root of a hermitian
    matrix M. i,e, R such that RR = M

    Parameters
    ----------
    M: array(m, m)
        Hermitian matrix

    Returns
    -------
    array(m, m): square-root matrix
    """
    evals, evecs = jnp.linalg.eigh(M)
    R = evecs @ jnp.sqrt(jnp.diag(evals)) @ jnp.linalg.inv(evecs)
    return R

In [1889]:
n_params, _ = all_params.shape
Sigma0 = jnp.eye(n_params)
Sigma_half = sqrtm(Sigma0)
Sigma_half

DeviceArray([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [1890]:
model.apply(params_reconstructed, x[1, None])

DeviceArray([[2.85888]], dtype=float32)

In [1891]:
shapes

[(3,), (1, 3), (1,), (3, 1)]

In [1892]:
treedef

PyTreeDef(CustomNode(<class 'flax.core.frozen_dict.FrozenDict'>[()], [{'params': {'layer1': {'bias': *, 'kernel': *}, 'layer_out': {'bias': *, 'kernel': *}}}]))

In [1893]:
def reconstruct_weights(params_flat, shapes, treedef):
    indices = jnp.array([np.prod(s) for s in shapes]).cumsum()
    split_reconstruction = jnp.array_split(params_flat, indices)
    split_reconstruction = [arr.reshape(shape) for arr, shape in zip(split_reconstruction, shapes)]
    params = jax.tree_unflatten(treedef, split_reconstruction)
    return params


def forward_mlp(X, params_flat, model, shapes, treedef):
    """
    Run the flax-MLP with flattened parameters
    
    Parameters
    ----------
    X: array(n_obs, dim_in)
        Array of observations
    params_flat: array(n_params, )
    model: Subclass of nn.module
    shapes: list of tuples
        Each element contains the original
        shape of the MLP
    treedef: PyTreeDef
        Result of running jax.tree_util.tree_flatten
        over `model`
    
    Returns
    -------
    * array(n_obs, dim_out?)
    """
    params = reconstruct_weights(params_flat, shapes, treedef)
    yhat = model.apply(params, X)
    return yhat

In [1894]:
forward_mlp(x[1, None],  all_params, model, shapes, treedef)

DeviceArray([[2.85888]], dtype=float32)

In [1895]:
model.apply(params, x[1, None])

DeviceArray([[2.85888]], dtype=float32)

In [1896]:
class UnscentedKalmanFilter:
    """
    Implementation of the Unscented Kalman Filter for discrete time systems
    """
    def __init__(self, fz, fx, Q, R, alpha, beta, kappa):
        self.fz = fz
        self.fx = fx
        self.Q = Q
        self.R = R
        self.d, _ = Q.shape
        self.alpha = alpha
        self.beta = beta
        self.kappa = kappa
        self.lmbda = alpha ** 2 * (self.d + kappa) - self.d
        self.gamma = jnp.sqrt(self.d + self.lmbda)
    
    @staticmethod
    def sqrtm(M):
        """
        Compute the matrix square-root of a hermitian
        matrix M. i,e, R such that RR = M
        
        Parameters
        ----------
        M: array(m, m)
            Hermitian matrix
        
        Returns
        -------
        array(m, m): square-root matrix
        """
        evals, evecs = jnp.linalg.eigh(M)
        R = evecs @ jnp.sqrt(jnp.diag(evals)) @ jnp.linalg.inv(evecs)
        return R

    def filter(self, init_state, sample_obs, X):
        """
        Run the Unscented Kalman Filter algorithm over a set of observed samples.
        Parameters
        ----------
        sample_obs: array(nsamples, obs_size)
        Returns
        -------
        * array(nsamples, state_size)
            History of filtered mean terms
        * array(nsamples, state_size, state_size)
            History of filtered covariance terms
        """
        wm_vec = jnp.array([1 / (2 * (self.d + self.lmbda)) if i > 0
                            else self.lmbda / (self.d + self.lmbda)
                            for i in range(2 * self.d + 1)])
        wc_vec = jnp.array([1 / (2 * (self.d + self.lmbda)) if i > 0
                            else self.lmbda / (self.d + self.lmbda) + (1 - self.alpha ** 2 + self.beta)
                            for i in range(2 * self.d + 1)])
        
        
        nsteps, _ = sample_obs.shape
        mu_t = init_state
        Sigma_t = self.Q

        mu_hist = jnp.zeros((nsteps, self.d))
        Sigma_hist = jnp.zeros((nsteps, self.d, self.d))

        mu_hist = index_update(mu_hist, 0, mu_t)
        Sigma_hist = index_update(Sigma_hist, 0, Sigma_t)
        
        for t in range(0, nsteps):
            # TO-DO: use jax.scipy.linalg.sqrtm when it gets added to lib
            comp1 = mu_t[:, None] + self.gamma * self.sqrtm(Sigma_t)
            comp2 = mu_t[:, None] - self.gamma * self.sqrtm(Sigma_t)
            
            sigma_points = jnp.concatenate((mu_t[:, None], comp1, comp2), axis=1)

            z_bar = self.fz(sigma_points)
            mu_bar = z_bar @ wm_vec
            Sigma_bar = (z_bar - mu_bar[:, None])
            Sigma_bar = jnp.einsum("i,ji,ki->jk", wc_vec, Sigma_bar, Sigma_bar) + self.Q
            
            # Second step
            Sigma_bar_half = self.sqrtm(Sigma_bar)
            comp1 = mu_bar[:, None] + self.gamma * Sigma_bar_half
            comp2 = mu_bar[:, None] - self.gamma * Sigma_bar_half
            #sigma_points = jnp.c_[mu_bar, comp1, comp2]
            sigma_points = jnp.concatenate((mu_bar[:, None], comp1, comp2), axis=1)

            ## ****** Some important code here ******
            x_bar = self.fx(X[t, None], sigma_points).ravel()
            x_hat = x_bar @ wm_vec
            St = (x_bar - x_hat)
            # St = jnp.einsum("i,ji,ki->jk", wc_vec, St, St) + self.R
            St = (wc_vec * St ** 2).sum() + self.R

            mu_hat_component = (z_bar - mu_bar[:, None])
            x_hat_component = (x_bar - x_hat)[None, :]
            Sigma_bar_y = jnp.einsum("i,ji,ki->jk", wc_vec, mu_hat_component, x_hat_component)
            
        
            Kt = Sigma_bar_y @ jnp.linalg.inv(St)
            mu_t = mu_bar + Kt @ (sample_obs[t] - x_hat)
            Sigma_t = Sigma_bar - Kt @ St @ Kt.T
            
            mu_t = mu_t.ravel()
            mu_hist = index_update(mu_hist, t, mu_t)
            Sigma_hist = index_update(Sigma_hist, t, Sigma_t)

        return mu_hist, Sigma_hist

In [1897]:
# Check whether 'unrolled' forward is the same as flax's forward
fx_p = partial(forward_mlp, model=model, shapes=shapes, treedef=treedef)
assert fx_p(x[1, None], all_params)  == model.apply(params, x[1, None])

In [1899]:
n_params, _ = all_params.shape

key = PRNGKey(314)
σ = 0.1
mu0 = normal(key, (n_params,)) * σ
mu0 = jnp.zeros((n_params, ))

In [1914]:
Q = jnp.eye(n_params) * σ ** 2
R = jnp.eye(1) * 0.9
alpha, beta, kappa = 0.001, 2, len(all_params)
alpha, beta, kappa = 1.0, 2.0, 3 - len(all_params)

fx_p = partial(forward_mlp, model=model, shapes=shapes, treedef=treedef)
fx_vmap = jax.vmap(fx_p, in_axes=[None, 1])
ukf = UnscentedKalmanFilter(fz, fx_vmap, Q, R, alpha, beta, kappa)
mu_hist, Sigma_hist = ukf.filter(mu0, y[:, None], x)

In [1915]:
xtest = jnp.linspace(x.min(), x.max(), 200)
yhat_test = model.apply(params, xtest)

yhat_ukf = fx_p(xtest, mu_hist[-1])
plt.plot(xtest, yhat_ukf, label=f"UKF+MLP ($\sigma^2={σ ** 2:0.3}$ | single epoch)", linewidth=2)
plt.plot(xtest, yhat_test, c="tab:red", label=f"SGD+MLP ({n_epochs} epochs)", linewidth=2)
plt.plot(xtest, f(xtest), label="underlying", linestyle="--")
plt.scatter(x, y, s=14, c="none", edgecolor="black", label="observations", alpha=0.3)
plt.legend()

In [1916]:
jnp.mean((f(xtest) - yhat_ukf.ravel()) ** 2)

DeviceArray(9.4379, dtype=float32)

In [1917]:
jnp.mean((f(xtest) - yhat_test.ravel()) ** 2)

DeviceArray(9.90347, dtype=float32)

In [1918]:
all_params.ravel()

DeviceArray([-10.06236, -11.09292,   6.91377,   5.49244,  -4.63497,
               4.14535,  -1.87585,  11.97218,  -7.16364,   7.72491],            dtype=float32)

In [1919]:
jnp.diag(Sigma_hist[0])

DeviceArray([0.02   , 0.02   , 0.02   , 0.02   , 0.02   , 0.02   ,
             0.01978, 0.02   , 0.02   , 0.02   ], dtype=float32)

In [1920]:
jnp.diag(Sigma_hist[-1])

DeviceArray([0.54281, 0.5029 , 0.34827, 0.2417 , 0.20957, 0.88479,
             0.53779, 0.51038, 0.29802, 0.17828], dtype=float32)

In [1921]:
np.set_printoptions(suppress=True, precision=5)

In [1922]:
reconstruct_weights(mu_hist[-1], shapes, treedef)

FrozenDict({
    params: {
        layer1: {
            bias: DeviceArray([-5.18205,  4.36082,  0.47092], dtype=float32),
            kernel: DeviceArray([[-2.94358, -2.44377, -4.26166]], dtype=float32),
        },
        layer_out: {
            bias: DeviceArray([0.77002], dtype=float32),
            kernel: DeviceArray([[-15.33198],
                         [-15.73477],
                         [  5.07862]], dtype=float32),
        },
    },
})

In [1923]:
params

FrozenDict({
    params: {
        layer1: {
            bias: DeviceArray([-10.06236, -11.09292,   6.91377], dtype=float32),
            kernel: DeviceArray([[ 5.49244, -4.63497,  4.14535]], dtype=float32),
        },
        layer_out: {
            bias: DeviceArray([-1.87585], dtype=float32),
            kernel: DeviceArray([[11.97218],
                         [-7.16364],
                         [ 7.72491]], dtype=float32),
        },
    },
})

In [1925]:
import matplotlib.animation as animation

nframes = n_obs
save_animation = True
fig, ax = plt.subplots()
def func(i):
    plt.cla()
    ax.plot(xtest, fx_p(xtest, mu_hist[i, :, None]), label=f"UKF+MLP ($\sigma^2={σ ** 2:0.2f}$ | single epoch)")
    ax.plot(xtest, yhat_test, c="tab:red", label=f"SGD+MLP ({n_epochs} epochs)")
    ax.plot(xtest, f(xtest), label="underlying", linestyle="--")
    ax.scatter(x[:i], y[:i], s=14, c="none", edgecolor="black", label="observations")
    ax.scatter(x[i], y[i], s=30, c="tab:red")
    ax.legend(loc="upper left")
    return ax
    
ani = animation.FuncAnimation(fig, func, frames=nframes)

#writer = animation.FFMpegFileWriter(bitrate=500)
if save_animation:    
    ani.save("/Users/gerardoduran/Desktop/cov_ukf4.mp4", dpi=100, bitrate=-1)

In [1393]:
def construct_params(params_flat, shapes, treedef):
    indices = jnp.array([np.prod(s) for s in shapes]).cumsum()
    split_reconstruction = jnp.array_split(params_flat, indices)
    split_reconstruction = [arr.reshape(shape) for arr, shape in zip(split_reconstruction, shapes)]
    params = jax.tree_unflatten(treedef, split_reconstruction)
    return params

In [932]:
ukf_params = construct_params(mu_hist[-1], shapes, treedef)

In [933]:
loss(ukf_params)

DeviceArray(67.17924, dtype=float32)

In [806]:
ukf = UnscentedKalmanFilter(fz, fx_vmap, Sigma_hist[-1,...], R, alpha, beta, kappa)
mu_hist, Sigma_hist = ukf.filter(mu_hist[-1, :], y[:, None], x)

In [649]:
def forward_mlp(X, params_flat, model, shapes, treedef):
    indices = jnp.array([np.prod(s) for s in shapes]).cumsum()
    split_reconstruction = jnp.array_split(params_flat, indices)
    split_reconstruction = [arr.reshape(shape) for arr, shape in zip(split_reconstruction, shapes)]
    params = jax.tree_unflatten(treedef, split_reconstruction)
    yhat = model.apply(params, X)
    return yhat

In [348]:
params_test = all_params * jnp.ones((1, 283))
fx_p(x[1, None], params_test[:, 1])

DeviceArray([[-25.98435]], dtype=float32)

In [350]:
params_test.shape

(22, 283)

In [349]:
fx_vmap = jax.vmap(fx_p, in_axes=[None, 1])
fx_vmap(x[1, None], params_test).shape

(283, 1, 1)

# EKF method

## Jacobian of the MLP

In [284]:
class D_MLP1D(nn.Module):
    n_hidden: int
        
    def setup(self):
        self.model = MLP1D(self.n_hidden)
    
    def __call__(self, x):
        return jax.jacfwd(self.model)(x)

In [295]:
X = normal(key2, (100, 2))

In [296]:
key = PRNGKey(314)
key1, key2 = split(key, 2)


n_hidden = 10
model = MLP1D(n_hidden)
D_model = MLP1D(n_hidden)

params = model.init(key1, X)
yhat = model.apply(params, X)

In [303]:
x.shape

(100,)

In [301]:
D_model.apply(params, X).shape

(100, 1, 1)

## References
### UKF + MLP
* https://ieeexplore.ieee.org/document/6234549
* https://juniperpublishers.com/raej/pdf/RAEJ.MS.ID.555568.pdf
### Flax
* https://github.com/google/flax/issues/735
* https://flax.readthedocs.io/en/latest/notebooks/flax_basics.html